# Industry Accelerators - SME Banking Prediction Models

## Introduction
In this notebook we will be going through an end-to-end project to load data, build four machine learning models, and then save and deploy models. Our pre_processed data is a data frame that is the clean data of loan attributes.

We are going to build four machine learning models in the following order: 

* **acceptance_classifier**: to predict if a loan is fully paid or charged off.
* **loan_life_classifier**: to predict if the life of a loan is more than a year or not.
* **credit_limit_model**: to predict the amount of loan given to a customer. This model incorporates the results of the first two models.
* **interest_rate_model**: to predict the interest rate of the loan. 

For every model, we go through the following process:

1. Identify the features and target columns
2. Balancing and splitting data
3. Model training
4. Model performance
5. Feature importance

And for two models (loan_life_classifier and interest_rate_model) we save and deploy them in the deployment space.

For the other two models, there is a documentation in `README` on how to train and deploy these models with AutoAI.

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020, 2021. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.<br>**

## Step 0: If you are running on Cloud Pak for Data as a Service (Public Cloud)
A pre-requisite of running the rest of the cells is to have:

- From the "More" menu above, select the "Insert project token" option
- Enter your "API Key" and "location" (e.g. 'us-south') in the cell below
- From the top level menu, select "Deployments"->"View all Spaces", select "New Deployment Space" with the name "SMEBanking", selecting an Object Store and Machine Learning Service.

In [ ]:
# If running on Cloud, enter your API key and instance location below.
api_key = ''
location = 'us-south'

## Setup connection to Machine Learning Client and target Deployment Space

In [ ]:
from ibm_watson_machine_learning import APIClient
import os

if api_key != '':
    wml_credentials = {
        "apikey": api_key,
        "url": 'https://' + location + '.ml.cloud.ibm.com'
    }
else:
    token = os.environ['USER_ACCESS_TOKEN']

    wml_credentials = {
        "token": token,
        "instance_id" : "openshift",
        "url": os.environ['RUNTIME_ENV_APSX_URL'],
        "version": "3.5"
     }

client = APIClient(wml_credentials)

### Enable Hyper parameter optimization

In [ ]:
# Set True if HPO is needed, else set False
enable_hyperparameter_tuning = False 

## Check for existance of the target deployment space and create if not already created

In [ ]:
# Specify a name for the target deployment space
space_name = 'SMEBanking'

# set space ID
space_uid = ''

# Check is the space already exists
for space in client.spaces.get_details()['resources']:
    if space['entity']['name'] == space_name:
        print("Deployment space",space_name,"already exists, using the existing space")
        space_uid=space['metadata']['id']
        client.set.default_space(space_uid)
        break

# if space doesn't exist then create it
if space_uid == '':
    print("Creating a new deployment space -",space_name)
    space_meta_data = {
        client.spaces.ConfigurationMetaNames.NAME : space_name
    }
    stored_space_details = client.spaces.store(space_meta_data)
    space_uid = stored_space_details['metadata']['id']
    client.set.default_space(space_uid)

## Load Pre-processed Data
For this project we will be loading pre_processed data by calling `training-data.csv`. 

The easiest way to load in data is to use the `Find and Add Data` icon in the upper right hand corner. Once selected you will see a sidebar come out with options to load from either local, remote, or other.

If you loaded your dataset into an Analytics Project, like a CSV file, then select Local and you should be able to find your dataset name. From there you can click the `Insert to code` and select to either insert a pandas dataframe or a spark dataframe. Once you make the selection you'll see python code inserted into the notebook cell with either Pandas or PySpark code for reading in your data. Now you're ready to explore and manipulate your dataset.

In [ ]:
# When running on Cloud click the option "Insert project token" - if not running on cloud project is setup below
try:
    project
except NameError:
    # READING AND WRITING PROJECT ASSETS
    import project_lib
    project = project_lib.Project() 

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# load prepared data 
df = pd.read_csv(project.get_file('training-data.csv'))
# display the pre_processed data
print('\nFirst 5 rows of the dataset:')
display(df.head())
print("The dataset has {:,} rows, {} columns".format(*df.shape))

In [ ]:
drop_col=['business_id', 'business_name', 'address', 'business_postcode', 'business_address_moveIn', 
        'business_tel', 'business_UStaxId', 'business_ownerName', 'business_naics_industry']
print('Drop these columns for the subsequent model training efforts:\n', drop_col)

df = df.drop(columns=drop_col)

### Identify the features and target columns

The `training_data.csv` data contains the clean data of customers, and `analytics` is the list of potential target variables to exclude or consider as the target variable.

We prepare the main variables for training the first model. `features_accept` is the list of features, and `target_accept` is the target variable.

In [ ]:
# set of analytical quantities for target variables
analytics = ['LoanStatus',
             'LifeLongerThanOneYear',
             'Yield',
             'funded_amount',
             'loan_interest_rate']

# identify the features columns for acceptance_classifier model
features_accept = [col for col in df.columns if col not in analytics]
target_accept = 'LoanStatus'

### Splitting data
After we set up the appropriate `features` and `target` variables then we split our data into train and test data sets using 90:10 split with 90% of the data being used for training our model and 10% being held out to test or validate our model.

Also, we set the seed using `random_state` parameter for reproducibility purposes.

In [ ]:
#!pip install sklearn==0.23 -q
import sklearn
print(sklearn.__version__)
from sklearn.model_selection import train_test_split

def train_test(df, features, target):
    """
    A function to split data to train and test set.

    Parameters
    ----------
    :param df: the entire data frame.

    :param features: list of features for the train set.

    :param target: name of target variable as string.

    Returns
    -------
    :return: X_train, X_test.
    """

    idx_train, idx_test = train_test_split(np.arange(df.shape[0]), test_size=0.1, random_state = 101) 
    
    X_train = df.iloc[idx_train,:]
    X_test  = df.iloc[idx_test, :]
    
    return X_train, X_test

In [ ]:
# splitting data into train and test sets
X_train_accept, X_test_accept = train_test(df, features_accept, target_accept)

## Building Machine Learning Models

 The final phase, and the most fun part, is the modeling. We'll train and validate four machine learning models:<br>
 
 1.  XGBClassifier for loan acceptance classifier 
 2.  XGBClassifier for loan life classifier
 3.  XGBRRegressor for credit limit regressor
 4.  XGBRegressor for interest rate regressor
 
For each of these models we'll train our model and test the model showing evaluation metrics for both the training and testing data. Then we'll look at the confusion matrix, to see where the model is classifying best across the binary labels. We also plot the distribution of residuals for the regression model. Finally we'll look at the variable importance plots. 

Let's take a look at training these models and their accuracies.

**One of the reasons we chose the prediction models from XGBoost family is because of their high performance, portability and compatibility with different platforms. However, the notebook also supports implementation of other machine learning models.



## 1. Loan Acceptance classifier

First step is to build the binary classification model of loans that classifies the loans into fully paid / charged off. 
For this purpose, we fit the classifier using Bayesian Optimization. This method that builds a probability model of the objective function, is proven to be more efficient than classical grid search and random search methods while achieve better performance

In [ ]:
# import modeling libraries
import sys
!{sys.executable} -m pip install bayesian-optimization
from sklearn.model_selection import cross_val_score

### Model training 

### Hyper Parameter Optimization (HPO) function

In [ ]:
#XGBoost hyper-parameter tuning
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.018],
        'max_depth': [10, 45],
        'subsample': [0.7, 0.81],
        'n_estimators' : [250, 457]
    }

    xgb_model = XGBClassifier()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_

### Model Building with or without Hyper parameter optimization 

In [ ]:
from xgboost import XGBClassifier
if enable_hyperparameter_tuning == True:
    print ('Initializing hyper-parameter optimization...')
    # HPO execution
    best_params=hyperParameterTuning(X_train_accept[features_accept], X_train_accept[target_accept])
    learning_rate=best_params['learning_rate']
    max_depth=best_params['max_depth']
    n_estimators=best_params['n_estimators']
    subsample=best_params['subsample']
    
    print ('Optimal params: ',best_params)
    print('Initializing model building process with hyper-parameter optimization')
    #Initialize an XGBClassifier with the tuned parameters and fit the training data
    acceptance_classifier = XGBClassifier(
                                    max_depth=max_depth,
                                    learning_rate=learning_rate,
                                    n_estimators=n_estimators,
                                    subsample=subsample,
                                    random_state=21)
else:
    print('Initializing model building process without hyper-parameter optimization')
    acceptance_classifier = XGBClassifier(
                                max_depth=45,
                                learning_rate=0.018,
                                n_estimators=457,
                                subsample=0.81,
                                random_state=21)
    
acceptance_classifier.fit(X_train_accept[features_accept], X_train_accept[target_accept])

print ('Model building successful')

### Model performance

We evaluate the model performance by measuring AUC and accuracy scores on the test set and getting the classification report for each class. The **print_score** function can be used to calculate the same metrics for the training data as well.

The accuracy and AUC look sufficiently good, but we also want to see where the model is misclassifying across our binary labels. For that we also show the confusion matrix.

Then, we calculate the cross validation score with **cross_validate_score** function.

In [ ]:
# Model evaluation metrics
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

def print_score(model, X_train, y_train, X_test, y_test, train=True, learning_type='classification'):
    """
    A function to compute the evaluation metrics for classification or regression models.

    Parameters
    ----------
    :param model: the classification or regression model.

    :param X_train: training set.

    :param y_train: training target.
    
    :param X_test: testing set.

    :param y_test testing target.

    :param learning_type: indicates if the model type is classification or regression.

    Returns
    -------
    :print: evaluation metrics for classification or regression models.
    """
    
    y_pred = model.predict(X_test)
    X_pred = model.predict(X_train)
    
    # evaluation metrics for classification
    if learning_type=='classification':
        if train:
            print("Train Result:\n")
            print("AUC: {0:.4f}\n".format(roc_auc_score(y_train, X_pred)))
            print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, X_pred)))
            print("Classification Report: \n {}\n".format(classification_report(y_train, X_pred)))
            print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, X_pred)))
        
        elif train==False:
            print("Test Result:\n")   
            print("AUC score: {0:.4f}\n".format(roc_auc_score(y_test, model.predict_proba(X_test)[:,1])))
            print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, y_pred)))
            print("Classification Report: \n {}\n".format(classification_report(y_test, y_pred)))
            print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, y_pred))) 
    
    # evaluation metrics for regression
    elif learning_type=='regression':   
        def mape(y_true, y_pred): 
            y_true, y_pred = np.array(y_true), np.array(y_pred)
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

        print("R2:",r2_score(y_test, y_pred))
        print("MAE:",mae(y_test, y_pred))
        print("MSE:",mse(y_test, y_pred))
        print("MAPE:",mape(y_test, y_pred))

In [ ]:
# display evaluation metrics
print_score(acceptance_classifier, 
            X_train_accept[features_accept], X_train_accept[target_accept], 
            X_test_accept[features_accept],   X_test_accept[target_accept], train=False, learning_type='classification')

In [ ]:
# Cross validation score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold, KFold

# cross_validate model using training data
def cross_validate_score(model, X_train, y_train, learning_type, optimization_metric, cv_num_folds, random_state):
    """
    A function to compute the cross validation score over training set.

    Parameters
    ----------
    :param model: the classification or regression model.

    :param X_train: training set.

    :param y_train: training target.

    :param learning_type: indicates if the model type is classification or regression.
    
    :param optimization_metric: the evaluation metric used in cross validation.

    :param cv_num_folds: the number of cross validation folds.

    :param random_state: random state.

    Returns
    -------
    :return: mean of cross validation scores.
    """
    if learning_type == 'classification':
        fold_generator = StratifiedKFold(n_splits=cv_num_folds, random_state=random_state)
    else:
        fold_generator = KFold(n_splits=cv_num_folds, random_state=random_state)
    cv_results = cross_validate(model, X_train, y_train, cv=fold_generator, scoring={optimization_metric:'roc_auc'}, return_train_score=True)
    print("Cross validation score:")
    return np.mean(cv_results['test_' + optimization_metric])

In [ ]:
# Cross validation score
cross_validate_score(model= acceptance_classifier, 
                    X_train= X_train_accept[features_accept],  
                    y_train= X_train_accept[target_accept], 
                    learning_type = 'classification', 
                    optimization_metric= 'roc_auc', 
                    cv_num_folds = 5, 
                    random_state = 30)

### Feature importance

With decision tree based models you can view variable importance plots. These plots indicate, at the feature level, some importance for the variable to predict the outcome variable. It uses gini importance which looks at node impurity. Node impurity is just looking at how well a tree splits the data.

Let's take a look at the feature importance plots and the list of features ranked by their importance, provided by **importance** function. 

In [ ]:
#feature importance
import matplotlib.pyplot as plt

def importance(model, df_X, features):
    """
    A function to feature ranking and plot feature importance of XGBoost.
    
    Parameters
    ----------
    :param model: the XGBoost model.

    :param df_X: the X_train with corresponding features.
    
    :param features: set of features for the training set.

    Prints
    -------
    :Prints: feature ranking.
    :Plots: feature importance.
    """
    feature_importances = pd.DataFrame(model.feature_importances_, 
                                   index = features,
                                   columns=['importance']).\
                                   sort_values('importance',ascending=False)

    # Print the feature ranking
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    print("Feature ranking:")
    for f in range(0,10):
        print("%d. feature %s (%f)" % (f + 1, df_X.columns[indices[f]], importances[indices[f]]))
            
    # Plot
    feature_importances.iloc[:10,:].plot(kind='barh')
    plt.show()

In [ ]:
#feature importance
importance(acceptance_classifier, X_train_accept[features_accept], features_accept)

### Save and deploy model

We save and deploy the model by connecting to the Watson Machine Learning service using CP4D credentials.
Watson Machine Learning provides deployment spaces where the user can save, configure and deploy their models. We can save models, functions and data assets in this space. The steps involved in saving and deploying the model are detailed in the following cells. We will use the watson_machine_learning_client package to complete these steps.

* Connect to WML client
* Save the model in the deployment space repository
* Deploy the model ONLINE

#### Save the model in the deployment space repository 

In [ ]:
# Select Software specification to deploy the models. Ensuring a matching environment between this notebook and the deployment will avoid an error loading the 
# model with an 'XGBoostLabelEncoder' attribute missing. 
#
# This will list the specifications (environments)
#client.software_specifications.list()
#
#sw_spec_uid = client.software_specifications.get_uid_by_name("default_py3.7")
sw_spec_uid = client.software_specifications.get_uid_by_name("default_py3.7_opence")

In [ ]:
# give model name
model = acceptance_classifier
model_name = 'acceptance_classifier'
X_test, y_test = X_test_accept[features_accept], X_test_accept[target_accept]

# get the fields from features to use when storing the model
fields = []
for feature in features_accept:
    fields.append( {'name': feature, 'type': 'double', 'nullable': False, 'metadata': {}} )
    
# Store the model details
model_props = {client.repository.ModelMetaNames.NAME: model_name,
               client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid,
               client.repository.ModelMetaNames.TYPE : "scikit-learn_0.23",
               # Indicating the input data type gives access to the form input type when testing the model through the GUI 
               client.repository.ModelMetaNames.INPUT_DATA_SCHEMA : {'id': '1', 'type': 'struct', 'fields': fields}
               }

# store model in the deployment space
stored_model_details = client.repository.store_model(model=model, meta_props=model_props, training_data=pd.DataFrame(X_test), training_target=pd.DataFrame(y_test), feature_names = list(X_test.columns))

#### Deploy the model ONLINE

In [ ]:
# deployment metadata of the model
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: model_name,
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# deploy the model
model_uid = stored_model_details["metadata"]["id"]
deployment_details = client.deployments.create( artifact_uid=model_uid, meta_props=meta_props)

## 2. Loan life classifier

Second step is to build the binary classification model for life of loans that classifies the loans into longer than one year / less than one year. We again fit the classifier with Bayesian Optimization. 

### Identify the features and target columns

In [ ]:
# identify the features columns for loan_life model
features_life = [col for col in df.columns if col not in analytics]+['funded_amount']
target_life = 'LifeLongerThanOneYear'

### Splitting data

In [ ]:
X_train_life, X_test_life = train_test(df, features_life, target_life)

### Model training

In [ ]:
from xgboost import XGBClassifier
if enable_hyperparameter_tuning == True:
    print ('Initializing hyper-parameter optimization...')
    # HPO execution
    best_params=hyperParameterTuning(X_train_life[features_life], X_train_life[target_life])
    learning_rate=best_params['learning_rate']
    max_depth=best_params['max_depth']
    n_estimators=best_params['n_estimators']
    subsample=best_params['subsample']
    
    print ('Optimal params: ',best_params)
    print('Initializing model building process with hyper-parameter optimization')
    #Initialize an XGBClassifier with the tuned parameters and fit the training data
    loan_life_classifier = XGBClassifier(
                                    max_depth=max_depth,
                                    learning_rate=learning_rate,
                                    n_estimators=n_estimators,
                                    subsample=subsample,
                                    random_state=21)
else:
    print('Initializing model building process without hyper-parameter optimization')
    loan_life_classifier = XGBClassifier(
                                max_depth=45,
                                learning_rate=0.018,
                                n_estimators=457,
                                subsample=0.81,
                                random_state=21)
    
loan_life_classifier.fit(X_train_life[features_life], X_train_life[target_life])

print ('Model building successful')

### Model performance

In [ ]:
# display evaluation metrics
print_score(loan_life_classifier, 
            X_train_life[features_life], X_train_life[target_life], 
            X_test_life[features_life],   X_test_life[target_life], train=False, learning_type='classification')

In [ ]:
# Cross validation score
cross_validate_score(model= loan_life_classifier, 
                    X_train= X_train_life[features_life],  
                    y_train= X_train_life[target_life],  
                    learning_type = 'classification', 
                    optimization_metric= 'roc_auc', 
                    cv_num_folds = 3, 
                    random_state = 30)

### Feature importance

In [ ]:
#feature importance
importance(loan_life_classifier, X_train_life[features_life], features_life)

### Save and deploy model

We save and deploy the model by connecting to the Watson Machine Learning service using CP4D credentials.
Watson Machine Learning provides deployment spaces where the user can save, configure and deploy their models. We can save models, functions and data assets in this space. The steps involved in saving and deploying the model are detailed in the following cells. We will use the watson_machine_learning_client package to complete these steps.

* Connect to WML client
* Save the model in the deployment space repository
* Deploy the model ONLINE

#### Save the model in the deployment space repository

In [ ]:
# give model name
model = loan_life_classifier
model_name = 'loan_life_classifier'
X_test, y_test = X_test_life[features_life], X_test_life[target_life]

# get the fields from features to use when storing the model
fields = []
for feature in features_life:
    fields.append( {'name': feature, 'type': 'double', 'nullable': False, 'metadata': {}} )
    
# Store the model details
model_props = {client.repository.ModelMetaNames.NAME: model_name,
               client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid,
               client.repository.ModelMetaNames.TYPE : "scikit-learn_0.23",
               # Indicating the input data type gives access to the form input type when testing the model through the GUI 
               client.repository.ModelMetaNames.INPUT_DATA_SCHEMA : {'id': '1', 'type': 'struct', 'fields': fields}
               }

# store model in the deployment space
stored_model_details = client.repository.store_model(model=model, meta_props=model_props, training_data=pd.DataFrame(X_test), training_target=pd.DataFrame(y_test), feature_names = list(X_test.columns))

#### Deploy the model ONLINE

In [ ]:
# deployment metadata of the model
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: model_name,
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# deploy the model
model_uid = stored_model_details["metadata"]["id"]
deployment_details = client.deployments.create( artifact_uid=model_uid, meta_props=meta_props)

## 3. Credit Limit regressor

Third step is to build the regression model for credit limit that predicts the amount of loan suggested. 

This model takes the result of `acceptance_classifier` and `loan_life_classifier` and predicts the credit limit only for the loans that are predicted as fully paid and their life longer than one year.

### Score the previous models 

In [ ]:
# score data as binary and probability    
df['LoanStatus_predicted']= acceptance_classifier.predict(df[features_accept])
df['LoanStatus_proba']= acceptance_classifier.predict_proba(df[features_accept])[:, 1]

df['LifeLongerThanOneYear_predicted']= loan_life_classifier.predict(df[features_life])
df['LifeLongerThanOneYear_proba']= loan_life_classifier.predict_proba(df[features_life])[:, 1]

### Identify the features and target columns

In [ ]:
# subset data for only for the loans that are predicted as fully paid and their life longer than one year
df_cr = df[(df['LifeLongerThanOneYear_predicted']==1) & (df['LoanStatus_predicted']==0) & (df['Yield']>0)]

In [ ]:
# Identify the features columns
analytics = analytics + ['LoanStatus_predicted', 'LifeLongerThanOneYear_predicted']
features_credit = [col for col in df.columns if col not in analytics]+['funded_amount']
target_credit = 'Yield'

In [ ]:
# Save data for AutoAI
data_credit_autoai = df_cr[features_credit + ['Yield']]

project.save_data("creditlimit-data-autoai.csv", data_credit_autoai.to_csv(index=False), overwrite=True)

### Splitting data

In [ ]:
X_train_credit, X_test_credit = train_test(df_cr, features_credit, target_credit)

### Model training 

In [ ]:
#Initialize an XGBRegressor with the tuned parameters and fit the training data
from xgboost import XGBRegressor
credit_limit = XGBRegressor(boosting_type='gbdt', num_leaves=1200,
                                learning_rate=0.01, n_estimators=500, max_depth=10,
                                metric='rmse', bagging_fraction=0.8, feature_fraction=0.8, random_state=21)
credit_limit.fit(X_train_credit[features_credit], X_train_credit[target_credit])

### Model performance

In [ ]:
# display evaluation metrics
print_score(credit_limit, 
            X_train_credit[features_credit], X_train_credit[target_credit], 
            X_test_credit[features_credit],   X_test_credit[target_credit], train=False, learning_type='regression')

### Residual plots

In [ ]:
# distribution plots of residuals
import matplotlib.pyplot as plt
import seaborn as sns
y_pred_credit = credit_limit.predict(X_test_credit[features_credit])
err=X_test_credit[target_credit]-y_pred_credit
plt.hist(np.abs(err),bins=100,cumulative=True);
plt.show()
sns.distplot(err,bins=100);

In [ ]:
# plot of predicted versus true values
f = plt.figure(figsize=[10,10])
sns.regplot(x=X_test_credit[target_credit], y=y_pred_credit);
plt.show()

### Feature importance

In [ ]:
importance(credit_limit, X_train_credit[features_credit], features_credit)

### Save and deploy model

We save and deploy the model by connecting to the Watson Machine Learning service using CP4D credentials.
Watson Machine Learning provides deployment spaces where the user can save, configure and deploy their models. We can save models, functions and data assets in this space. The steps involved in saving and deploying the model are detailed in the following cells. We will use the watson_machine_learning_client package to complete these steps.

* Connect to WML client
* Save the model in the deployment space repository
* Deploy the model ONLINE

#### Save the model in the deployment space repository

In [ ]:
# give model name
model = credit_limit
model_name = 'credit_limit'
X_test, y_test = X_test_credit[features_credit], X_test_credit[target_credit]

# get the fields from features to use when storing the model
fields = []
for feature in features_credit:
    fields.append( {'name': feature, 'type': 'double', 'nullable': False, 'metadata': {}} )
    
# Store the model details
model_props = {client.repository.ModelMetaNames.NAME: model_name,
               client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid,
               client.repository.ModelMetaNames.TYPE : "scikit-learn_0.23",
               # Indicating the input data type gives access to the form input type when testing the model through the GUI 
               client.repository.ModelMetaNames.INPUT_DATA_SCHEMA : {'id': '1', 'type': 'struct', 'fields': fields}
               }

# store model in the deployment space
stored_model_details = client.repository.store_model(model=model, meta_props=model_props, training_data=pd.DataFrame(X_test), training_target=pd.DataFrame(y_test), feature_names = list(X_test.columns))

#### Deploy the model ONLINE

In [ ]:
# deployment metadata of the model
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: model_name,
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# deploy the model
model_uid = stored_model_details["metadata"]["id"]
deployment_details = client.deployments.create( artifact_uid=model_uid, meta_props=meta_props)

## 4. Interest Rate regressor

The fourth step is to build the regression model for predicting the interest rate of the loan. 

### Identify the features and target columns

In [ ]:
# identify the features columns for acceptance_classifier model
analytics = analytics + ['LoanStatus_proba', 'LifeLongerThanOneYear_proba']
features_rate = [col for col in df.columns if col not in analytics]
target_rate = 'loan_interest_rate'

### Splitting data

In [ ]:
X_train_rate, X_test_rate = train_test(df, features_rate, target_rate)

### Model training

In [ ]:
#Initialize an XGBRegressor with the tuned parameters and fit the training data
from xgboost import XGBRegressor
interest_rate = XGBRegressor(boosting_type='gbdt', num_leaves=1200,
                                learning_rate=0.01, n_estimators=500, max_depth=10,
                                metric='rmse', bagging_fraction=0.8, feature_fraction=0.8, random_state=21)
interest_rate.fit(X_train_rate[features_rate], X_train_rate[target_rate])

### Model performance

In [ ]:
# display evaluation metrics
print_score(interest_rate, 
            X_train_rate[features_rate], X_train_rate[target_rate], 
            X_test_rate[features_rate],   X_test_rate[target_rate], train=False, learning_type='regression')

### Feature importance

In [ ]:
importance(interest_rate, X_train_rate[features_rate], features_rate)

### Save model
Please make sure that the user input and connection to WML are set.
#### Save the model in the deployment space repository

In [ ]:
# give model name
model = interest_rate
model_name = 'interest_rate'
X_test, y_test = X_test_rate[features_rate], X_test_rate[target_rate]

In [ ]:
# get the fields from features to use when storing the model
fields = []
for feature in features_rate:
    fields.append( {'name': feature, 'type': 'double', 'nullable': False, 'metadata': {}} )

In [ ]:
# Store the model details
model_props = {client.repository.ModelMetaNames.NAME: model_name,
               client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid,
               client.repository.ModelMetaNames.TYPE : "scikit-learn_0.23",
               # Indicating the input data type gives access to the form input type when testing the model through the GUI 
               client.repository.ModelMetaNames.INPUT_DATA_SCHEMA : {'id': '1', 'type': 'struct', 'fields': fields}
               }

# store model in the deployment space
stored_model_details = client.repository.store_model(model=model, meta_props=model_props, training_data=pd.DataFrame(X_test), training_target=pd.DataFrame(y_test), feature_names = list(X_test.columns))

#### Deploy the model ONLINE

In [ ]:
# deployment metadata of the model
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: model_name,
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# deploy the model
model_uid = stored_model_details["metadata"]["id"]
deployment_details = client.deployments.create( artifact_uid=model_uid, meta_props=meta_props)